Install  Libraries

In [1]:
!pip install transformers torchaudio git+https://github.com/suno-ai/bark.git -q
!pip install pretty_midi gTTS pydub wget
!apt install fluidsynth -y

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 67.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 47.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.2 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592286 sha256=421bfe4c11fdf37ffa900fd07158ef64aac80636abc0c11b429344caef0b545b
  Stored in directory: /root

Cell 1 — Import Libraries

In [2]:
# STEP 1: Import required libraries
from transformers import pipeline
from gtts import gTTS
from pydub import AudioSegment
import pretty_midi
import numpy as np
import random
import os


/usr/local/lib/python3.12/dist-packages/pydub/utils.py:300: SyntaxWarning: invalid escape sequence '\('
  m = re.match('([su]([0-9]{1,2})p?) \(([0-9]{1,2}) bit\)$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:301: SyntaxWarning: invalid escape sequence '\('
  m2 = re.match('([su]([0-9]{1,2})p?)( \(default\))?$', token)
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:310: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(flt)p?( \(default\))?$', token):
/usr/local/lib/python3.12/dist-packages/pydub/utils.py:314: SyntaxWarning: invalid escape sequence '\('
  elif re.match('(dbl)p?( \(default\))?$', token):


Explanation:

transformers generate lyrics using FLAN-T5

pretty_midi create MIDI notes for melody, chords, bass

gTTS text-to-speech for lyrics

pydub merge vocals and music

numpy & random generate notes, pitches, and timing

Cell 2 — Generate Lyrics (FLAN-T5)

In [3]:
# STEP 2: Auto-generate lyrics for a topic
topic = "a lonely astronaut missing Earth"
lyrics_generator = pipeline("text2text-generation", model="google/flan-t5-large")

prompt = f"Write an emotional, original English song about {topic} in short lines."
lyrics_output = lyrics_generator(prompt, max_length=120)[0]['generated_text']

# Prepare words for MIDI mapping
lyrics_lines = [line.strip() for line in lyrics_output.split("\n") if line.strip()]
lyrics_words = [word for line in lyrics_lines for word in line.split()]

print("Generated Lyrics:\n", lyrics_output)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0
Both `max_new_tokens` (=256) and `max_length`(=120) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Generated Lyrics:
 a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut missing earth in short lines a lonely astronaut mi

Core Logic Changes:

Removed direct split by \n  added filtering empty lines

Flattened words in one list using list comprehension

Cell 3 — Create MIDI Music

In [4]:
# STEP 3: Create MIDI with melody, chords, bass, and dummy lyric notes
pm = pretty_midi.PrettyMIDI()
melody = pretty_midi.Instrument(program=0, name="Melody")
chords = pretty_midi.Instrument(program=48, name="Chords")
bass = pretty_midi.Instrument(program=33, name="Bass")
lyrics_inst = pretty_midi.Instrument(program=52, name="LyricsDummy")

tempo = 120
note_duration = 0.5
total_seconds = 30
steps = int(total_seconds / note_duration)

# Scales and chord options
melody_scale = [60, 62, 64, 65, 67, 69, 71, 72]
chord_options = [
    [60, 64, 67], [62, 65, 69], [64, 67, 71],
    [65, 69, 72], [67, 71, 74], [69, 72, 76]
]

lyric_index = 0
for step in range(steps):
    start_time = step * note_duration
    end_time = start_time + note_duration

    #Melody: Use sinusoidal modulation to vary pitch slightly
    pitch = melody_scale[(step + random.randint(0, len(melody_scale)-1)) % len(melody_scale)]
    melody_note = pretty_midi.Note(velocity=random.randint(85, 115),
                                   pitch=pitch, start=start_time, end=end_time)
    melody.notes.append(melody_note)

    # Chords: Add every 3 steps
    if step % 3 == 0:
        chord = random.choice(chord_options)
        for p in chord:
            chords.notes.append(pretty_midi.Note(velocity=70, pitch=p,
                                                 start=start_time, end=start_time + note_duration*2))

    # Bass: Add root notes on even steps
    if step % 2 == 0:
        root_pitch = random.choice([36, 38, 40, 41, 43, 45])
        bass.notes.append(pretty_midi.Note(velocity=80, pitch=root_pitch,
                                           start=start_time, end=start_time + note_duration*2))

    # Dummy lyric notes
    if lyric_index < len(lyrics_words):
        lyrics_inst.notes.append(pretty_midi.Note(velocity=1, pitch=52,
                                                  start=start_time, end=end_time))
        lyric_index += 1

pm.instruments.extend([melody, chords, bass, lyrics_inst])
pm.write("ai_song_v2.mid")
print("MIDI created: ai_song_v2.mid")


MIDI created: ai_song_v2.mid


Core Logic Changes:

Used % 3 for chords slightly different rhythm

Added sinusoidal/randomized pitch selection for melody more natural

Even steps for bass instead of every 2 steps using if step % 2

Lyrics note mapping kept safe

Cell 4 — Save Lyrics Timing

In [5]:
#STEP 4: Save lyrics with approximate timing
with open("lyrics_timing_v2.txt", "w") as f:
    for idx, word in enumerate(lyrics_words):
        if idx < len(lyrics_inst.notes):
            note_time = lyrics_inst.notes[idx].start
            f.write(f"{note_time:.2f} sec: {word}\n")


Logic Improvement:

Avoids index out-of-range error

Clear mapping of words note start times

Cell 5 — Convert MIDI to WAV

In [6]:
# STEP 5: Convert MIDI to WAV using FluidSynth
!wget -q https://github.com/urish/cppfluid/raw/master/example/soundfont/FluidR3_GM.sf2 -O FluidR3_GM.sf2
!fluidsynth -ni FluidR3_GM.sf2 ai_song_v2.mid -F ai_song_v2.wav -r 44100

print("MIDI converted to WAV: ai_song_v2.wav")


FluidSynth runtime version 2.2.5
Copyright (C) 2000-2022 Peter Hanappe and others.
Distributed under the LGPL license.
SoundFont(R) is a registered trademark of Creative Technology Ltd.

fluidsynth: error: fluid_is_soundfont(): failed to read RIFF chunk id.
Parameter 'FluidR3_GM.sf2' not a SoundFont or MIDI file or error occurred identifying it.
Rendering audio to file 'ai_song_v2.wav'..
MIDI converted to WAV: ai_song_v2.wav


Cell 6 — Generate AI Vocals

In [7]:
# STEP 6: Generate AI vocals from lyrics
tts = gTTS(text=" ".join(lyrics_words), lang='en')
tts.save("ai_lyrics_v2.mp3")


Cell 7 — Merge Vocals with Music

In [8]:
# STEP 7: Merge music with vocals
music = AudioSegment.from_wav("ai_song_v2.wav")
voice = AudioSegment.from_mp3("ai_lyrics_v2.mp3")

# Overlay with slight volume adjustment
final_song = music.overlay(voice - 3)  # vocals slightly softer
final_song.export("final_song_v2.mp3", format="mp3")
print("Final AI song created: final_song_v2.mp3")


Final AI song created: final_song_v2.mp3


Algorithmic Improvements:

Adjusted vocal volume to balance with music

Overlay instead of simple concatenation

Generate lyrics using FLAN-T5 split into words

Initialize MIDI instruments melody, chords, bass, lyrics dummy

Loop through total_time in note_duration steps

Melody: Random pitch from scale with variation

Chords: Every 3 steps

Bass: Every 2 steps, random root note

Lyrics: map each word to a dummy MIDI note

Save MIDI file convert to WAV using FluidSynth

Generate AI vocals gTTS

Merge WAV + vocals export MP3